### Prometheus example

![](https://prometheus-operator.dev/img/architecture.png)

Quelle: [Blogeintrag](https://prometheus-operator.dev/docs/user-guides/getting-started/)

- - -

Prometheus provides an example of a monitored service.

This is described in this [Blogeintrag](https://prometheus-operator.dev/docs/user-guides/getting-started/).

We start the example or the microservice and create a `ServiceMonitor` at the same time

In [ ]:
%%bash
kubectl create namespace mon-app

cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice   
spec:
  replicas: 3
  selector:
    matchLabels:
      app: example-app
  template:
    metadata:
      labels:
        app: example-app
        tier: microservice        
    spec:
      containers:
      - name: example-app
        image: fabxc/instrumented_app
        ports:
        - name: web
          containerPort: 8080
---
kind: Service
apiVersion: v1
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice    
spec:
  selector:
    app: example-app
  ports:
  - name: web
    port: 8080
  type:
    LoadBalancer
---    
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: microservice
  labels:
    tier: microservice
spec:
  selector:
    matchLabels:
      tier: microservice
  endpoints:
  - port: web
%EOF%


And check if all pods have been started

In [ ]:
! kubectl get all,servicemonitors --namespace mon-app -l tier=microservice

If so, we can query the Metrics data

In [ ]:
! curl example-app.mon-app:8080/metrics

- - -
# Deploy Prometheus

We can now monitor the created microservices with Prometheus.

For this we need the appropriate rights:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: mon-app
%EOF%


And a separate Prometheus instance.

With

      alerting:  
        alertmanagers:
        - apiVersion: v2
          name: alertmanager-example
          namespace: mon-app
          pathPrefix: /
          port: web
          
we establish the connection to the alert manager.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: v1
kind: Service
metadata:
  name: prometheus
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: LoadBalancer
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  alerting:  
    alertmanagers:
    - apiVersion: v2
      name: alertmanager-example
      namespace: mon-app
      pathPrefix: /
      port: web
  ruleSelector:
    matchLabels:
      role: alert-rules
      prometheus: prometheus      
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
%EOF%

In the user interface we can then look at the monitored targets and their status.

To do this, enter `up` as a query or display the monitored microservices via pulldown -> Status -> Targets.

Also, we can get all the values ??from the command `curl example-app.mon-app:8080/metrics`, too.

In [ ]:
! echo "Prometheus UI  : http://$(cat ~/work/server-ip):$(kubectl get service --namespace mon-app prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"

---

# [Bereitstellung Alertmanager](https://prometheus-operator.dev/docs/user-guides/alerting/)

The Prometheus operator introduces an Alertmanager resource that allows users to declaratively describe an Alertmanager cluster. To successfully deploy an Alertmanager cluster, it is important to understand how Prometheus interacts with the Alertmanager.

The Alertmanager is used for:
* Deduplicate alerts received from Prometheus.
* Mute warnings.
* Forward notifications to different endpoints (PagerDuty, OpsGenie, Mail, Chat,…).



In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: monitoring.coreos.com/v1
kind: Alertmanager
metadata:
  name: example
spec:
  replicas: 1
  alertmanagerConfigSelector:
    matchLabels:
      alertmanagerConfig: example
---
apiVersion: v1
kind: Service
metadata:
  name: alertmanager-example
spec:
  type: NodePort
  ports:
  - name: web
    port: 9093
    protocol: TCP
    targetPort: web
  selector:
    alertmanager: example
%EOF%

With `AlertmanagerConfig` we can tell the alert manager how to react to alerts.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: monitoring.coreos.com/v1alpha1
kind: AlertmanagerConfig
metadata:
  name: config-example
  labels:
    alertmanagerConfig: example
spec:
  route:
    groupBy: ['job']
    groupWait: 30s
    groupInterval: 5m
    repeatInterval: 12h
    receiver: 'webhook'
  receivers:
  - name: 'webhook'
    webhookConfigs:
    - url: 'http://example.com/'
%EOF%

And with `PrometheusRule` the corresponding rule is activated in Prometheus

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace mon-app -f -
apiVersion: monitoring.coreos.com/v1
kind: PrometheusRule
metadata:
  creationTimestamp: null
  labels:
    prometheus: prometheus
    role: alert-rules    
  name: prometheus-example-rules
spec:
  groups:
  - name: example-app
    rules:
    - alert: ExampleAlert
      expr: vector(1)
%EOF%


## control

In the Prometheus UI, under the pulldowns
* Status -> Runtime Information & Build Information an Alertmanager Endpoint must be available.
* Status -> Three instances of the example app
* Status -> Rules die Example-App Rule
And at `Alerts` a notification.

A notification can also be seen in the Alertmanager UI.

In [ ]:
! echo "Prometheus   UI: http://$(cat ~/work/server-ip):$(kubectl get service --namespace mon-app prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"
! echo "Alertmanager UI: http://$(cat ~/work/server-ip):$(kubectl -n mon-app get service alertmanager-example -o=jsonpath='{ .spec.ports[0].nodePort }')"

---
### Clean up

In [ ]:
! kubectl delete namespace mon-app